# Retail Performance Analysis — Complete Notebook

This notebook performs a full retail analytics pipeline on a sample Superstore-like dataset.

### Files used (local paths)
- Dataset (example): `/mnt/data/retail_project_outputs/synthetic_superstore.csv`
- Project brief / ideas: `/mnt/data/data analyst projects.pdf`

If you have your real dataset, replace the `DATA_PATH` variable below with your CSV path.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os

# Paths (edit DATA_PATH to point to your real dataset if needed)
DATA_PATH = r"C:\Users\sahha\Downloads\Retail_Performance_Pharma_Notebook.ipynb"
PROJECT_PDF = r"/mnt/data/data analyst projects.pdf"
OUTPUT_DIR = Path('/mnt/data/retail_project_outputs/notebook_outputs')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print('Notebook outputs will be saved to', OUTPUT_DIR)


Notebook outputs will be saved to \mnt\data\retail_project_outputs\notebook_outputs


## 1) Load data

Load the CSV and inspect the first rows.

In [2]:
df = pd.read_csv(DATA_PATH, parse_dates=['Date'], low_memory=False)
# وبعدها أنشئ الأعمدة المتوقعة:
df['Order Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Ship Date'] = pd.NaT

# قراءة الملف (مع تحويل عمود Date لتاريخ)
DATA_PATH = r"C:\Users\sahha\Desktop\New folder (3)\pharmacy_sales_usa.csv"  # عدّل المسار لو لازم

import pandas as pd
df = pd.read_csv(DATA_PATH, parse_dates=['Date'], low_memory=False)

# اجعل العمود بنفس الاسم اللي النوتبوك متوقعه
df['Order Date'] = pd.to_datetime(df['Date'], errors='coerce')

# لو الكود بعد كده يتعامل مع Ship Date، نعرّف عمود فارغ (NaT) لتجنّب الأخطاء
df['Ship Date'] = pd.NaT

# عرض عيّنة للتأكد
print("Columns now:", list(df.columns))
df.head()

print('Rows, cols:', df.shape)
df.head()


ValueError: Missing column provided to 'parse_dates': 'Date'

## 2) Quick data quality checks

Check columns, missing values and duplicates.

In [ ]:
print('Columns:', list(df.columns))
print('\nMissing values per column:')
print(df.isna().sum())
print('\nNumber of duplicate rows:', df.duplicated().sum())


## 3) Basic cleaning & feature engineering

Create time features and profitability metrics.

In [ ]:
df = df.drop_duplicates().copy()
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month
df['Quarter'] = df['Order Date'].dt.to_period('Q')
df['Profit Margin'] = (df['Profit'] / df['Sales']).replace([np.inf, -np.inf], np.nan).fillna(0)
df['Sales_per_unit'] = (df['Sales'] / df['Quantity']).replace([np.inf, -np.inf], np.nan).fillna(0)
df['Discount_flag'] = (df['Discount'] > 0).astype(int)
print('After cleaning — rows, cols:', df.shape)
df.head()


## 4) KPI calculations

Compute a small KPI table.

In [ ]:
total_revenue = df['Sales'].sum()
total_profit = df['Profit'].sum()
avg_margin = (total_profit / total_revenue) if total_revenue else 0
num_orders = df['Order ID'].nunique() if 'Order ID' in df.columns else len(df)
num_products = df['Product Name'].nunique() if 'Product Name' in df.columns else 0
kpis = {
    'Total Revenue': total_revenue,
    'Total Profit': total_profit,
    'Avg Profit Margin': avg_margin,
    'Num Orders': num_orders,
    'Num Products': num_products
}
kpis


## 5) Monthly sales & profit trend

Plot monthly sales and profit series and save images to OUTPUT_DIR.

In [ ]:
monthly = df.groupby(pd.Grouper(key='Order Date', freq='M'))[['Sales','Profit']].sum().reset_index().sort_values('Order Date')
plt.figure(figsize=(10,4))
plt.plot(monthly['Order Date'], monthly['Sales'], marker='o')
plt.title('Monthly Sales Trend')
plt.xlabel('Month')
plt.ylabel('Sales')
plt.tight_layout()
monthly_sales_png = OUTPUT_DIR / 'monthly_sales.png'
plt.savefig(str(monthly_sales_png))
plt.show()

plt.figure(figsize=(10,4))
plt.plot(monthly['Order Date'], monthly['Profit'], marker='o')
plt.title('Monthly Profit Trend')
plt.xlabel('Month')
plt.ylabel('Profit')
plt.tight_layout()
monthly_profit_png = OUTPUT_DIR / 'monthly_profit.png'
plt.savefig(str(monthly_profit_png))
plt.show()


## 6) Category sales

Bar chart of sales by Category.

In [ ]:
cat_sales = df.groupby('Category')['Sales'].sum().sort_values(ascending=False)
plt.figure(figsize=(6,4))
cat_sales.plot(kind='bar')
plt.title('Sales by Category')
plt.ylabel('Sales')
plt.tight_layout()
category_sales_png = OUTPUT_DIR / 'category_sales.png'
plt.savefig(str(category_sales_png))
plt.show()


## 7) Discount vs Profit scatter (sample)

Scatter to visualise the relationship between discount and profit.

In [ ]:
sample = df.sample(n=min(1200, len(df)), random_state=1)
plt.figure(figsize=(6,4))
plt.scatter(sample['Discount'], sample['Profit'], alpha=0.6, s=10)
plt.title('Discount vs Profit (sample)')
plt.xlabel('Discount')
plt.ylabel('Profit')
plt.tight_layout()
disc_profit_png = OUTPUT_DIR / 'discount_vs_profit.png'
plt.savefig(str(disc_profit_png))
plt.show()


## 8) Top products

Horizontal bar of top 10 products by sales.

In [ ]:
top_products = df.groupby('Product Name')['Sales'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(8,4))
top_products.plot(kind='barh')
plt.title('Top 10 Products by Sales')
plt.xlabel('Sales')
plt.gca().invert_yaxis()
plt.tight_layout()
top_products_png = OUTPUT_DIR / 'top_products.png'
plt.savefig(str(top_products_png))
plt.show()


## 9) Region performance

Summarise sales and profit by region.

In [ ]:
region_perf = df.groupby('Region')[['Sales','Profit']].sum().sort_values('Sales', ascending=False)
region_perf


## 10) Summary exports

Save summary CSVs and list files created.

In [ ]:
summary = df.groupby('Category').agg({'Sales':'sum','Profit':'sum','Quantity':'sum'}).reset_index()
summary['Profit Margin'] = summary['Profit'] / summary['Sales']
summary_csv = OUTPUT_DIR / 'category_summary.csv'
summary.to_csv(summary_csv, index=False)
print('Saved', summary_csv)
print('Saved images:')
for p in ['monthly_sales.png','monthly_profit.png','category_sales.png','discount_vs_profit.png','top_products.png']:
    print('-', OUTPUT_DIR / p)


## Next steps / notes
- Replace `DATA_PATH` with your real dataset to reproduce results on real data.
- The project brief PDF is at the local path:
  - `/mnt/data/data analyst projects.pdf`
- Use the generated images in `/mnt/data/retail_project_outputs/notebook_outputs` for README or PPTX.
- If you want, copy this notebook into your repo `notebooks/` folder and run it in Jupyter to produce outputs.
